# 第10章数据聚合与分组操作

####  pandas提供一个灵活的groupby接口，允许你以一种自然的方式对数据集进行切片，切块，总结。
#### 在本章你将学到:
#### （1）使用一个或多个键（以函数，数组或dataframe列名的形式）将pandas对象拆分为多块
#### （2）计算组汇总统计信息，如计数，平均值或标准差或用户定义函数
#### （3）应用组内变换或其他操作，如标准化，线性回归，排位或子集选择
#### （4）计算数据透视表和交叉表
#### （5）执行分位数分析和其他统计组分析

## 10.1GroupBy 机制 

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})

In [4]:
df

,key1,key2,data1,data2
0,a,one,0.715843,-0.862354
1,a,two,-0.442626,0.075267
2,b,one,0.867231,0.098806
3,b,two,0.028995,0.772886
4,a,one,1.034547,-0.615303


#### 假设你想要根据key1标签计算data1列的均值，有很多方法可以实现。其中一种是访问data1并使用key1列（它是一个series）调用groupby方法。 

In [5]:
grouped=df['data1'].groupby(df['key1'])

In [6]:
grouped

####  grouped 变量现在是一个GroupBy对象。除了一些关于分组键df['key'] 的一些中间数据之外，它实际上还没进行任何计算。这个对象拥有 所有必需信息，之后可以在每个分组上应用一些操作。例如，为了计算分组的均值我们可以调用GroupBy的mean方法：

In [8]:
grouped.mean()

key1
a    0.435921
b    0.448113
Name: data1, dtype: float64

####  之后，我们将更多地解释调用.mean()发生了什么。数据（一个series）根据分组键进行了聚合，并产生了新的Series,这个Series使用key1列唯一值作为索引。由于DataFrame的列df['key1'],结果中的索引名称是‘key1’ 。
#### 如果我们将多个数组作为列表df['key1'],则我们会得到一些不同的结果：

In [11]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()

In [12]:
means

key1  key2
a     one     0.875195
      two    -0.442626
b     one     0.867231
      two     0.028995
Name: data1, dtype: float64

In [43]:
type(means)

pandas.core.series.Series

#### 这里我们使用了两个键对数据进行分组，并且结果Series现在拥有一个包含唯一键对的多层索引：

In [13]:
type(means)

pandas.core.series.Series

In [14]:
means.unstack()

key2,one,two
key1,,
a,0.875195,-0.442626
b,0.867231,0.028995


#### 在这个例子中，分组键都是Series,尽管分组键也可以是正确长度的任何数组：

In [15]:
states=np.array(['Ohio','California','California','Ohio','Ohio'])

In [16]:
years=np.array([2005,2005,2006,2005,2006])

In [17]:
df['data1'].groupby([states,years]).mean()

California  2005   -0.442626
            2006    0.867231
Ohio        2005    0.372419
            2006    1.034547
Name: data1, dtype: float64

#### 分组信息作为你想要继续处理的数据，通常包含在同一个DataFrame中。在这种情况下，你可以传递列名（无论那些列名是字符串，数据或其他Python对象）作为分组键： 

In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.435921,-0.467463
b,0.448113,0.435846


In [20]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.875195 -0.738828
     two  -0.442626  0.075267
b    one   0.867231  0.098806
     two   0.028995  0.772886

#### 如果不在意使用groupby的目的，通用的GroupBy方法是size,size方法返回一个包含组大小的信息的Series: 

In [21]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

####  请注意，分组键中的任何缺失值将被排除在结果之外。

### 10.1.1 遍历各分组

#### GroupBy对象支持迭代，会生成一个包含组名和数据块2维元组序列。考虑一下代码：

In [25]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.715843 -0.862354
1    a  two -0.442626  0.075267
4    a  one  1.034547 -0.615303
b
  key1 key2     data1     data2
2    b  one  0.867231  0.098806
3    b  two  0.028995  0.772886


#### 在多个分组键的情况下，元组中的第一个元素是键值的元组： 

In [27]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.715843 -0.862354
4    a  one  1.034547 -0.615303
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.442626  0.075267
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.867231  0.098806
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.028995  0.772886


#### 当然你可以选择在任何一块数据上进行你想要的操作。使用一行代码计算出数据块的字典可能会对你有用 

In [28]:
pieces=dict(list(df.groupby('key1')))

In [29]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.715843 -0.862354
 1    a  two -0.442626  0.075267
 4    a  one  1.034547 -0.615303, 'b':   key1 key2     data1     data2
 2    b  one  0.867231  0.098806
 3    b  two  0.028995  0.772886}

In [30]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one  0.715843 -0.862354
  1    a  two -0.442626  0.075267
  4    a  one  1.034547 -0.615303), ('b',   key1 key2     data1     data2
  2    b  one  0.867231  0.098806
  3    b  two  0.028995  0.772886)]

In [31]:
df.groupby('key1')

In [32]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.867231,0.098806
3,b,two,0.028995,0.772886


In [33]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [135]:
grouped=df.groupby(df.dtypes,axis=1)

In [136]:
list(grouped)

[(dtype('float64'),       data1     data2
  0  0.715843 -0.862354
  1 -0.442626  0.075267
  2  0.867231  0.098806
  3  0.028995  0.772886
  4  1.034547 -0.615303), (dtype('O'),   key1 key2
  0    a  one
  1    a  two
  2    b  one
  3    b  two
  4    a  one)]

In [141]:
grouped1=df.groupby(df.dtypes)

In [142]:
list(grouped1)

[]

In [37]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.715843 -0.862354
1 -0.442626  0.075267
2  0.867231  0.098806
3  0.028995  0.772886
4  1.034547 -0.615303
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2选择一列或所有列的子集 

####  将从DataFrame 创建的GroupBy 对象用列名称或列名称数组进行索引时，会产生用于聚合的列子集的效果。这表明：

#### 在处理数据集时，要计算data2列的均值，并获得DataFrame形式的结果，我们可以写：

In [49]:
m2=df[['data2']].groupby([df['key1'],df['key2']]).mean()

In [44]:
m1=df.groupby(['key1','key2'])[['data2']].mean()

In [46]:
type(m1)

pandas.core.frame.DataFrame

In [50]:
type(m2)

pandas.core.frame.DataFrame

In [51]:
m1

data2
key1 key2          
a    one  -0.738828
     two   0.075267
b    one   0.098806
     two   0.772886

In [52]:
m2

data2
key1 key2          
a    one  -0.738828
     two   0.075267
b    one   0.098806
     two   0.772886

####  如果传递的是列表或数组，则此索引操作返回的对象是分组的DataFrame;如果只有单个列名作为标量传递，则为分组的Series:

In [61]:
m3=df['data2'].groupby([df['key1'],df['key2']]).mean()

In [63]:
m4=df.groupby([df['key1'],df['key2']])['data2'].mean()   # 写成[['key1'],['key2']]也可以

In [55]:
m3

key1  key2
a     one    -0.738828
      two     0.075267
b     one     0.098806
      two     0.772886
Name: data2, dtype: float64

In [64]:
m4

key1  key2
a     one    -0.738828
      two     0.075267
b     one     0.098806
      two     0.772886
Name: data2, dtype: float64

In [57]:
type(m3)

pandas.core.series.Series

In [58]:
type(m4)

pandas.core.series.Series

### 10.3.1 使用字典和Series 分组 

In [98]:
people=pd.DataFrame(np.random.randn(5,5),columns=['a','b','c','d','e'],index=['Joe','Steve','Wes','Jim','Travis'])

In [99]:
people

,a,b,c,d,e
Joe,2.059265,1.169634,0.970892,-0.300690,-0.307074
Steve,0.034252,-1.102953,0.430506,0.702639,1.789784
Wes,1.005799,-0.045732,0.176908,-0.930162,-1.844057
Jim,-0.828626,0.703095,1.287721,-2.182915,0.523080
Travis,1.328233,0.925632,-1.336282,0.112124,-0.948523


In [100]:
people.iloc[2:3,[1,2]]=np.nan

In [101]:
people

,a,b,c,d,e
Joe,2.059265,1.169634,0.970892,-0.300690,-0.307074
Steve,0.034252,-1.102953,0.430506,0.702639,1.789784
Wes,1.005799,NaN,NaN,-0.930162,-1.844057
Jim,-0.828626,0.703095,1.287721,-2.182915,0.523080
Travis,1.328233,0.925632,-1.336282,0.112124,-0.948523


####  假设我拥有各列的分组对应关系，并且想要把各列按组累加：

In [102]:
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

In [103]:
by_column=people.groupby(mapping,axis=1)

In [104]:
list(by_column)

[('blue',                c         d
  Joe     0.970892 -0.300690
  Steve   0.430506  0.702639
  Wes          NaN -0.930162
  Jim     1.287721 -2.182915
  Travis -1.336282  0.112124), ('red',                a         b         e
  Joe     2.059265  1.169634 -0.307074
  Steve   0.034252 -1.102953  1.789784
  Wes     1.005799       NaN -1.844057
  Jim    -0.828626  0.703095  0.523080
  Travis  1.328233  0.925632 -0.948523)]

In [108]:
by_column1=people.groupby(mapping,axis=0)

In [109]:
# list(by_column1)  #

In [110]:
by_column.count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [111]:
by_column.sum()

,blue,red
Joe,0.670202,2.921825
Steve,1.133145,0.721083
Wes,-0.930162,-0.838258
Jim,-0.895194,0.397550
Travis,-1.224158,1.305342


####  Series 也有相同的功能，可以视为固定大小的映射：

In [112]:
map_series=pd.Series(mapping)

In [113]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [114]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


###  10.1.4 使用函数分组

In [122]:
key_list=['one','one','one','two','two']

In [123]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.236438,1.872729,2.258613,-3.413767,-1.628050
5,0.034252,-1.102953,0.430506,0.702639,1.789784
6,1.328233,0.925632,-1.336282,0.112124,-0.948523


####  将函数与数组，字典或Series进行混合并不困难，所有对象都会在内部转换为数组：

In [124]:
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  1.005799  1.169634  0.970892 -0.930162 -1.844057
  two -0.828626  0.703095  1.287721 -2.182915  0.523080
5 one  0.034252 -1.102953  0.430506  0.702639  1.789784
6 two  1.328233  0.925632 -1.336282  0.112124 -0.948523

###  10.1.5根据索引层级分组

In [125]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])

In [126]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [127]:
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)

In [128]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.084396 -0.481906  0.020571  0.544386 -0.316480
1     -0.312949  0.083091  1.001798 -0.841478 -0.654849
2     -1.682236  0.783950 -0.144383 -0.386665 -1.790793
3      1.595435 -0.180723 -0.990523 -0.160633 -0.516706

In [129]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2数据聚合

#### 数据聚合是指所有根据数组产生标量值的数据转换过程。之前已经使用一些聚合操作，mean,count,min,sum 等。 

In [143]:
df

,key1,key2,data1,data2
0,a,one,0.715843,-0.862354
1,a,two,-0.442626,0.075267
2,b,one,0.867231,0.098806
3,b,two,0.028995,0.772886
4,a,one,1.034547,-0.615303


In [144]:
grouped=df.groupby('key1')

In [145]:
grouped['data1'].quantile(0.9)

key1
a    0.970806
b    0.783407
Name: data1, dtype: float64

####  要使用你自己的聚合函数，需要将函数传递给aggregate或agg方法： 

In [146]:
def peak_to_peak(arr):
    return arr.max()-arr.min()

In [147]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.477174,0.937621
b,0.838236,0.674080


####  你可能会注意到一些方法，比如describe也是有效的，尽管严格来说它们并不是聚合函数

In [148]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.435921  0.777353 -0.442626  0.136608  0.715843  0.875195   
b      2.0  0.448113  0.592722  0.028995  0.238554  0.448113  0.657672   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.034547   3.0 -0.467463  0.485979 -0.862354 -0.738828 -0.615303   
b     0.867231   2.0  0.435846  0.476646  0.098806  0.267326  0.435846   

                          
           75%       max  
key1                      
a    -0.270018  0.075267  
b     0.604366  0.772886

###  逐列及多函数应用

In [149]:
tips=pd.read_csv('D:/data_analysis/bookcode/pydata-book/examples/tips.csv')

In [186]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


244

In [155]:
tips['tip_pct']=tips['tip']/tips['total_bill']  # 添加总帐单的小费比例

In [157]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [158]:
grouped=tips.groupby(['day','smoker'])  # 首先根据day和smoker来对tips 进行分组：

In [160]:
list(grouped)

[(('Fri', 'No'),      total_bill   tip smoker  day    time  size   tip_pct
  91        22.49  3.50     No  Fri  Dinner     2  0.155625
  94        22.75  3.25     No  Fri  Dinner     2  0.142857
  99        12.46  1.50     No  Fri  Dinner     2  0.120385
  223       15.98  3.00     No  Fri   Lunch     3  0.187735),
 (('Fri', 'Yes'),      total_bill   tip smoker  day    time  size   tip_pct
  90        28.97  3.00    Yes  Fri  Dinner     2  0.103555
  92         5.75  1.00    Yes  Fri  Dinner     2  0.173913
  93        16.32  4.30    Yes  Fri  Dinner     2  0.263480
  95        40.17  4.73    Yes  Fri  Dinner     4  0.117750
  96        27.28  4.00    Yes  Fri  Dinner     2  0.146628
  97        12.03  1.50    Yes  Fri  Dinner     2  0.124688
  98        21.01  3.00    Yes  Fri  Dinner     2  0.142789
  100       11.35  2.50    Yes  Fri  Dinner     2  0.220264
  101       15.38  3.00    Yes  Fri  Dinner     2  0.195059
  220       12.16  2.20    Yes  Fri   Lunch     2  0.180921
  221  

In [161]:
grouped_pct=grouped['tip_pct']

In [163]:
list(grouped_pct)

[(('Fri', 'No'), 91     0.155625
  94     0.142857
  99     0.120385
  223    0.187735
  Name: tip_pct, dtype: float64), (('Fri', 'Yes'), 90     0.103555
  92     0.173913
  93     0.263480
  95     0.117750
  96     0.146628
  97     0.124688
  98     0.142789
  100    0.220264
  101    0.195059
  220    0.180921
  221    0.259314
  222    0.223776
  224    0.117735
  225    0.153657
  226    0.198216
  Name: tip_pct, dtype: float64), (('Sat', 'No'), 19     0.162228
  20     0.227679
  21     0.135535
  22     0.141408
  23     0.192288
  24     0.160444
  25     0.131387
  26     0.149589
  27     0.157604
  28     0.198157
  29     0.152672
  30     0.151832
  31     0.136240
  32     0.199203
  33     0.118415
  34     0.183915
  35     0.149626
  36     0.122624
  37     0.181335
  38     0.123596
  39     0.159898
  40     0.139651
  57     0.056797
  59     0.139424
  64     0.150085
  65     0.156873
  66     0.150152
  68     0.099357
  70     0.163894
  71     0.175747
  74  

In [164]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

#### 如果你传递的是函数或者函数名的列表，你会获得一个列名是这些函数名的DataFrame:

In [165]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

####  如果你传递的是（name,function）元组的列表，每个元组的第一个元素将作为DataFrame的列名 

In [168]:
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

####  你可以指定应用到所有列上的函数列表或每一列上要应用的不同函数。假设我们想要计算tip_pct和total_bill列的三个相同的统计值： 

In [169]:
functions=['count','mean','max']

In [170]:
result=grouped['tip_pct','total_bill'].agg(functions)

In [171]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [172]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

####  和以前一样，可以传递具有自定义名称的元组列表： 

In [173]:
ftuples=[('Durchschnitt','mean'),('Abweichung',np.var)]

In [174]:
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

####  假设你想要将不同的函数应用到一个或多个列上。要实现这个功能，需要将含有列名与函数对应关系的字典传递给agg:

In [178]:
grouped.agg({'tip':'max','size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [179]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

#### 只有多个函数应用于至少一列时，DataFrame才有分层列 

### 10.2.2 返回不含行索引的聚合数据 

####  在前面所有的例子中，聚合数据返回时都是带有索引的，有时索引时分层的，由唯一的分组键联合形成。因为不是所有的情况都索引，所以在多数情况下你可以通过向groupby传递as_index=False 来禁用分组键作为索引行为：

In [180]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


##  应用：通用拆分-应用-联合

####  假设你想要按组选出小费百分比（tip-pct）最高的五组。首先，写一个可以在特定列中选出最大值所在行的函数：

In [182]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]    #-n  ?

In [183]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


####  现在如果我们按照smoker进行分组，之后调用apply,我们会得到一下结果：

In [184]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

#### top 函数在DataFrame 的每一行分组上被调用，之后使用pandas.concat将函数结果黏贴在一起，并使用分组名作为各组的标签。因此结果包含在一个分层索引，该分层索引的内部层级包含原DataFrame的索引值：

####  如果你除了向apply传递函数，还传递其他函数或关键字的话，你可以把这些放在函数后进行传递：

In [189]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

#### 你可能还记得我之前在GroupBy对象上调用了describe方法： 

In [190]:
result=tips.groupby('smoker')['tip_pct'].describe()

In [191]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [193]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

###  10.3.1压缩分组键

####  在之前的例子中，你可以看到所得到对象具有分组键所形成的分层索引以及每个原始对象的索引。你可以通过向groupby传递group_keys=False 来禁用这个功能。

In [194]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [195]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

###  10.3.2 分位数与桶分析 

####  你

In [196]:
frame=pd.DataFrame({'data1':np.random.randn(1000),'data2':np.random.randn(1000)})

In [197]:
frame

,data1,data2
0,1.868446,-1.623658
1,0.736393,-0.320360
2,-0.094858,-0.533122
3,0.565285,-0.023290
4,-0.697227,1.478947
5,0.376545,-1.112237
6,-0.005019,-1.005702
7,-0.208319,1.479550
8,-0.073337,0.773618
9,1.063495,0.185187


In [198]:
quartiles=pd.cut(frame.data1,4)

In [199]:
quartiles[:10]

0     (0.274, 1.892]
1     (0.274, 1.892]
2    (-1.345, 0.274]
3     (0.274, 1.892]
4    (-1.345, 0.274]
5     (0.274, 1.892]
6    (-1.345, 0.274]
7    (-1.345, 0.274]
8    (-1.345, 0.274]
9     (0.274, 1.892]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.97, -1.345] < (-1.345, 0.274] < (0.274, 1.892] < (1.892, 3.511]]

####  cut 返回的的Categorical 对象可以直接传递给groupby.所以我们可以计算出data2列的一个统计值集合，如下：

In [201]:
def get_stats(group):
    return{'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}

In [202]:
grouped=frame.data2

In [203]:
grouped

0     -1.623658
1     -0.320360
2     -0.533122
3     -0.023290
4      1.478947
5     -1.112237
6     -1.005702
7      1.479550
8      0.773618
9      0.185187
10    -0.262609
11    -0.182622
12    -0.385053
13    -0.624948
14    -2.114053
15    -0.587220
16     0.884553
17     0.139495
18     1.227476
19     0.546971
20     0.368316
21    -0.150567
22    -1.651747
23    -0.540144
24     1.215694
25     0.256383
26     2.162484
27    -0.670725
28     1.663525
29     0.393636
         ...   
970   -0.641000
971   -0.730712
972   -0.689237
973   -1.497506
974   -1.379471
975    0.715138
976    2.170829
977   -1.894635
978   -0.727360
979    0.623823
980    1.099053
981    1.187104
982   -0.920196
983    0.056909
984    0.467344
985   -1.051042
986    0.802778
987    0.968000
988   -1.851879
989   -0.704252
990   -0.399683
991   -0.582642
992    1.736208
993   -0.695604
994   -1.978860
995   -1.627410
996    1.392525
997   -0.620472
998    0.546460
999   -0.099084
Name: data2, Length: 100

In [204]:
grouped=frame.data2.groupby(quartiles)

In [206]:
list(grouped)

[(Interval(-2.97, -1.345, closed='right'), 15    -0.587220
  19     0.546971
  30    -0.447663
  39     0.825557
  55     1.709458
  78    -1.213294
  95    -0.173273
  107    1.934767
  115    0.134954
  123    0.463048
  139   -0.619624
  146   -0.571499
  150   -1.193367
  160   -0.546696
  168    0.913786
  178    0.206261
  186    0.180748
  208   -0.724064
  211   -0.103646
  226   -0.901960
  230   -0.972898
  231   -1.507812
  236    0.869378
  240    0.035846
  243    0.259138
  248    1.358985
  256   -0.150274
  259   -1.948696
  260   -0.008933
  264   -0.694514
           ...   
  727   -0.461835
  729    1.409740
  731   -0.527064
  736   -0.503948
  738   -0.281991
  739   -1.268013
  750    1.070766
  762   -0.599037
  779   -2.535700
  785   -0.321270
  801    0.344459
  822    0.264324
  823    0.570895
  830    0.523921
  838    1.283490
  841    0.189358
  862   -0.412382
  880    1.504209
  882   -0.261014
  894    0.974660
  901   -1.051218
  905   -0.398350
  909

In [208]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.97, -1.345]",100.0,2.389881,0.048095,-2.535700
"(-1.345, 0.274]",501.0,2.529338,0.020070,-2.703262
"(0.274, 1.892]",375.0,3.013840,0.051280,-3.591951
"(1.892, 3.511]",24.0,2.463856,0.256799,-2.003725


####  这些就是等长的桶。为了根据样本分位数计算出等大小的桶，则需要用qcut.我们将传递labels=False 来获得分位数数值：

In [211]:
grouping=pd.qcut(frame.data1,10)

In [212]:
grouping

0        (1.266, 3.511]
1        (0.543, 0.889]
2       (-0.21, 0.0528]
3        (0.543, 0.889]
4      (-0.868, -0.496]
5         (0.27, 0.543]
6       (-0.21, 0.0528]
7       (-0.21, 0.0528]
8       (-0.21, 0.0528]
9        (0.889, 1.266]
10       (0.0528, 0.27]
11       (1.266, 3.511]
12     (-1.334, -0.868]
13      (-0.496, -0.21]
14     (-0.868, -0.496]
15     (-2.964, -1.334]
16       (0.889, 1.266]
17      (-0.21, 0.0528]
18       (0.543, 0.889]
19     (-2.964, -1.334]
20       (1.266, 3.511]
21     (-1.334, -0.868]
22       (1.266, 3.511]
23        (0.27, 0.543]
24       (0.0528, 0.27]
25       (0.0528, 0.27]
26     (-1.334, -0.868]
27      (-0.21, 0.0528]
28        (0.27, 0.543]
29        (0.27, 0.543]
             ...       
970      (0.543, 0.889]
971     (-0.21, 0.0528]
972    (-1.334, -0.868]
973      (0.0528, 0.27]
974      (0.0528, 0.27]
975       (0.27, 0.543]
976      (0.543, 0.889]
977     (-0.21, 0.0528]
978     (-0.21, 0.0528]
979      (0.0528, 0.27]
980       (0.27,

In [216]:
grouping=pd.qcut(frame.data1,10,labels=False)

In [217]:
grouping

0      9
1      7
2      4
3      7
4      2
5      6
6      4
7      4
8      4
9      8
10     5
11     9
12     1
13     3
14     2
15     0
16     8
17     4
18     7
19     0
20     9
21     1
22     9
23     6
24     5
25     5
26     1
27     4
28     6
29     6
      ..
970    7
971    4
972    1
973    5
974    5
975    6
976    7
977    4
978    4
979    5
980    6
981    2
982    6
983    6
984    1
985    3
986    2
987    4
988    1
989    3
990    1
991    1
992    5
993    1
994    5
995    4
996    3
997    4
998    1
999    6
Name: data1, Length: 1000, dtype: int64

In [218]:
grouped=frame.data2.groupby(grouping)

In [219]:
grouped

In [220]:
grouped.apply(get_stats)

data1       
0      count    100.000000
       max        2.389881
       mean       0.048095
       min       -2.535700
1      count    100.000000
       max        2.162484
       mean       0.047515
       min       -2.637141
2      count    100.000000
       max        2.146018
       mean       0.148439
       min       -2.114053
3      count    100.000000
       max        2.529338
       mean      -0.023548
       min       -2.580471
4      count    100.000000
       max        2.127141
       mean       0.072665
       min       -2.098819
5      count    100.000000
       max        2.131842
       mean      -0.125660
       min       -2.703262
6      count    100.000000
       max        1.944256
       mean      -0.052127
       min       -2.263542
7      count    100.000000
       max        3.013840
       mean       0.303331
       min       -2.474212
8      count    100.000000
       max        1.960295
       mean       0.063243
       min       -2.055827
9      count   

In [221]:
grouped.apply(get_stats).unstack()  

,count,max,mean,min
data1,,,,
0,100.0,2.389881,0.048095,-2.535700
1,100.0,2.162484,0.047515,-2.637141
2,100.0,2.146018,0.148439,-2.114053
3,100.0,2.529338,-0.023548,-2.580471
4,100.0,2.127141,0.072665,-2.098819
5,100.0,2.131842,-0.125660,-2.703262
6,100.0,1.944256,-0.052127,-2.263542
7,100.0,3.013840,0.303331,-2.474212
8,100.0,1.960295,0.063243,-2.055827


#### 我们将在第12章介绍categorial类型

###  10.3.3示例 ：使用指定分组值填充缺失值

####  这里我们用平均值来填充NA值

In [222]:
s=pd.Series(np.random.randn(6))

In [223]:
s[::2]=np.nan

In [224]:
s

0         NaN
1   -0.089742
2         NaN
3   -0.892518
4         NaN
5   -0.401126
dtype: float64

In [225]:
s.fillna(s.mean())

0   -0.461129
1   -0.089742
2   -0.461129
3   -0.892518
4   -0.461129
5   -0.401126
dtype: float64

####  假设你需要填充值按组来变化。一个方法是对数据分组后使用apply 和一个在每个数据块上都调用fillna的函数。下面是一些将美国分为东部地区和西部地区的样本数据：

In [227]:
states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [228]:
group_key=['East']*4+['West']*4

In [229]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [230]:
data=pd.Series(np.random.randn(8),index=states)

In [231]:
data

Ohio          1.666062
New York      0.425939
Vermont      -2.959951
Florida       0.829000
Oregon        2.811718
Nevada        1.048761
California   -0.832346
Idaho        -0.455888
dtype: float64

####  让我们将数据中的一些值设置为缺失值：

In [232]:
data[['Vermont']]       #定位数组中的某个值

Vermont   -2.959951
dtype: float64

In [233]:
data[['Vermont','Nevada','Idaho']]

Vermont   -2.959951
Nevada     1.048761
Idaho     -0.455888
dtype: float64

In [234]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [235]:
data

Ohio          1.666062
New York      0.425939
Vermont            NaN
Florida       0.829000
Oregon        2.811718
Nevada             NaN
California   -0.832346
Idaho              NaN
dtype: float64

In [236]:
data.groupby(group_key).mean()

East    0.973667
West    0.989686
dtype: float64

####  我们使用分组的平均值来填充NA值，如下： 

In [238]:
fill_mean=lambda g:g.fillna(g.mean())

In [239]:
fill_mean

<function __main__.<lambda>(g)>

In [240]:
data.groupby(group_key).apply(fill_mean)

Ohio          1.666062
New York      0.425939
Vermont       0.973667
Florida       0.829000
Oregon        2.811718
Nevada        0.989686
California   -0.832346
Idaho         0.989686
dtype: float64

####  在另一种情况下，你可能已经在代码种为每个分组预定义了填充值。由于每个分组都有一个内置的name属性，我们可以这样使用： 

In [241]:
fill_values={'East':0.5,'West':-1}

In [242]:
fill_func=lambda g:g.fillna(fill_values[g.name])

In [243]:
data.groupby(group_key).apply(fill_func)

Ohio          1.666062
New York      0.425939
Vermont       0.500000
Florida       0.829000
Oregon        2.811718
Nevada       -1.000000
California   -0.832346
Idaho        -1.000000
dtype: float64

### 1.3.4示例：随机采样与排列

####  假设你想从大数据集中抽取随机样本，我们可以使用Series的sample 方法。为了演示，我们讲解一种构造一副英式扑克牌的方法。

In [244]:
suits=['H','S','C','D']

In [245]:
card_val=(list(range(1,11))+[10]*3)*4

In [246]:
card_val

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [247]:
base_names=['A']+list(range(2,11))+['J','K','Q']

In [260]:
base_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

In [264]:
cards=[]

In [265]:
for suit in ['H','S','C','D']:
    for num in base_names:
        cards.append(str(num)+suit)

In [266]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [267]:
len(cards)

52

In [268]:
deck=pd.Series(card_val,index=cards)

In [269]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

#### 现在基于我之前说，从这副牌里拿出五张牌可以写成：

In [270]:
def draw(deck,n=5):
    return deck.sample(n)

In [271]:
draw(deck)

3C    3
5H    5
4S    4
6S    6
8H    8
dtype: int64

####  假设你想要从每个花色中随机抽取两张牌。由于花色是牌名的最后两个字符，我们可以基于这点进行分组，并使用apply:

In [273]:
get_suit=lambda card : card[-1] #last letter is suit

In [274]:
deck.groupby(get_suit).apply(draw,n=2)

C  6C     6
   4C     4
D  JD    10
   8D     8
H  KH    10
   6H     6
S  2S     2
   3S     3
dtype: int64

#### 或者也可以写成：

In [276]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

9C      9
5C      5
10D    10
4D      4
5H      5
8H      8
3S      3
2S      2
dtype: int64

In [278]:
deck.groupby(get_suit).apply(draw,n=2)

C  10C    10
   QC     10
D  7D      7
   9D      9
H  10H    10
   6H      6
S  5S      5
   QS     10
dtype: int64

### 10.3.5示例：分组加权平均和相关性

####   加权平均是可以做到的，作为一个例子，我们使用一个包含分组键和权重值的数据集：

In [280]:
df=pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],'data':np.random.randn(8),'weights':np.random.rand(8)})

In [281]:
df

,category,data,weights
0,a,-0.094201,0.336474
1,a,1.303822,0.195466
2,a,0.017921,0.681140
3,a,1.280310,0.480892
4,b,-0.696406,0.026701
5,b,-0.263723,0.423349
6,b,-0.414557,0.168517
7,b,0.673332,0.350110


####  通过category 进行分组加权平均如下：

In [282]:
grouped=df.groupby('category')

In [283]:
get_wavg=lambda g:np.average(g['data'],weights=g['weights'])

In [284]:
grouped.apply(get_wavg)

category
a    0.502402
b    0.036791
dtype: float64

####  作为另一个例子，考虑一个从雅虎财经上获得的数据集，该数据集包括标普500（spx）符号和股票的收盘价 

In [285]:
close_px=pd.read_csv('D:/data_analysis/bookcode/pydata-book/examples/stock_px_2.csv',parse_dates=True,index_col=0)

In [286]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [288]:
close_px[-4:]   #最后4个

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### 现在要计算一个DataFrame,它包含标普指数（spx）每日收益的年度相关性（通过百分比变化计算）。作为实现方式，我们首先创建一个计算每列与‘spx’列成对关联的函数：

In [289]:
spx_corr=lambda x:x.corrwith(x['SPX'])

#### 之后，我们使用pct_change计算close-px 百分比的变化：

In [290]:
rets=close_px.pct_change().dropna()

In [291]:
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
2003-01-10,0.002725,0.001824,-0.013927,0.000000
2003-01-13,-0.005435,0.008648,-0.004134,-0.001412
2003-01-14,-0.002732,0.010379,0.008993,0.005830
2003-01-15,-0.010959,-0.012506,-0.013713,-0.014426
2003-01-16,0.012465,-0.016282,0.004519,-0.003942


#### 最后，我们按照年对百分比变化进行分组，可以使用单行函数从每个标签中提取每个datetime标签的year 属性： 

In [292]:
get_year=lambda x:x.year

In [293]:
by_year=rets.groupby(get_year)

In [294]:
by_year

In [295]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


####  你也可以计算内部列相关性。这里我们计算了苹果和微软的年度相关性： 

In [297]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 10.3.6示例：逐组线性回归 

#### 我们可以定义一下regress(回归)函数使用计量经济库，该函数对每个数据块执行普通最小二乘回归：

In [299]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1
    result=sm.OLS(Y,X).fit()
    return result.params

#### 现在要计算AAPL在SPX回报上的年度线性回归，执行以下代码：

In [302]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 数据透视表与交叉表

####  在小费数据集中，假设你想要计算一张在行方向上按day和smoker 排列的分组平均值

In [304]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

#### 这个功能也可以直接使用groupby实现。现在，假设我们只想在tip_pct和size上进行聚合，并根据time分组。我将把smoker放入表的列，而将day 放入表的行：

In [305]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

#### 我们可以通过传递margins=True 来扩充这个表来包含部分总计。这会添加ALL行和列标签，其中相应的值是单层中所有数据的分组统计值： 

In [307]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

#### 要使用不同的聚合函数时，将函数传递给aggfunc。例如，‘count’或者len将给出一张分组大小的交叉表（计数或出现频率）：

In [308]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

#### 某些情况下产生了空值或NA，你可能想要传递一个fill_value:

In [309]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 交叉表：crosstab 

#### 交叉表是数据透视表的一个特殊情况，计算的是分组中的概率。下面是个例子： 

In [327]:
lNationality=['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA']
lSample=[x for x in range(1,11)]
lHandedness=['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed','Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']
data=pd.DataFrame({'Sample':lSample,'Nationality':lNationality,'Handedness':lHandedness})

In [328]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


####  现在要计算分组中的频率，你可以使用pivot_table 来实现这个功能，但是用pandas.crosstable 函数更方便：

In [329]:
pd.crosstab(data.Nationality,data.Handedness,margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


#### crosstab 的前两个参数可是数组，Series 或数组的列表。在小费数据中可以这么做：

In [332]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244